# Conjunction Finder

In [ ]:
# !pip install --upgrade --no-deps --force-reinstall 'git+https://github.com/ESA-VirES/VirES-Python-Client@swarm_ab_conjunctions#egg=viresclient'

SERVER_URL = "https://staging.viresdisc.vires.services/ows"

In [ ]:
import os
import json
import datetime as dt
import numpy as np
import xarray as xr
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from tqdm import tqdm
from viresclient import SwarmRequest

import panel as pn
pn.extension('ace')
import hvplot.xarray
import holoviews as hv
import geoviews as gv
hv.extension('bokeh')

# import warnings
# warnings.filterwarnings('ignore')

## Build interactivity for finding conjunction times

In [ ]:
widgets = {
    "times": pn.widgets.DateRangeSlider(
        name="Time range",
        start=dt.datetime(2013, 1, 1),
        end=dt.datetime.now(),
        value=(dt.datetime(2020, 1, 1), dt.datetime.now()),
    ),
    "mission1": pn.widgets.Select(name="Mission 1", options=["Swarm"]),
    "mission2": pn.widgets.Select(name="Mission 2", options=["Swarm"]),
    "spacecraft1": pn.widgets.Select(name="Spacecraft 1", options=["A"]),
    "spacecraft2": pn.widgets.Select(name="Spacecraft 2", options=["B"]),
    "threshold": pn.widgets.FloatSlider(name="Angular separation (degrees)", start=0, end=10, step=0.1, value=1.0),
    "fetch-conjs": pn.widgets.Button(name="Find conjunctions ➡️", button_type="primary"),
    "conj-times": pn.widgets.Tabulator(height=400, width=150),
    "spinner": pn.indicators.LoadingSpinner(value=False, width=100, height=100)
}


In [ ]:
def update_conj_times(event):
    # Blank the Tabulator widget while loading
    widgets["conj-times"].value = None
    widgets["spinner"].value = True
    # Fetch conjunctions from VirES
    request = SwarmRequest(SERVER_URL)
    conjs = request.get_conjunctions(
        start_time=widgets["times"].value[0],
        end_time=widgets["times"].value[1],
        threshold=widgets["threshold"].value,
        spacecraft1=widgets["spacecraft1"].value,
        spacecraft2=widgets["spacecraft2"].value,
        mission1=widgets["mission1"].value,
        mission2=widgets["mission2"].value
    ).as_dataframe()
    # Update Tabulator widget with the table
    conjs = conjs.drop(columns="AngularSeparation")
    conjs.index.name = "Select times:"
    widgets["conj-times"].value = conjs
    widgets["spinner"].value = False

widgets["fetch-conjs"].on_click(update_conj_times)

In [ ]:
# pn.layout.Row(
#     pn.layout.Column(
#         widgets["times"],
#         widgets["threshold"],
#         pn.layout.Row(
#             widgets["mission1"],
#             widgets["spacecraft1"],
#             width=300
#         ),
#         pn.layout.Row(
#             widgets["mission2"],
#             widgets["spacecraft2"],
#             width=300
#         ),
#         widgets["fetch-conjs"],
#         widgets["spinner"],
#         width=300
#     ),
#     widgets["conj-times"],
# )

## Extend with geographic preview of positions

In [ ]:
class DataPreview:

    def __init__(self):
        self.preview_map = gv.Overlay(
            [gv.feature.ocean.opts(alpha=0.7),
            gv.feature.land.opts(alpha=0.7)]
        )
        self.data = {"sc1": None, "sc2": None}
        empty_points = {"Latitude": np.array([]), "Longitude": np.array([])}
        self.preview_map*=gv.Points(empty_points).opts(color="blue")
        self.preview_map*=gv.Points(empty_points).opts(color="red")
        
    def display(self):
        return self.preview_map.opts(projection=ccrs.Robinson(), aspect=2, global_extent=True)

    def get_preview_data(
            self,
            mission="Swarm",
            spacecraft="A",
            time=np.datetime64("2020-01-01"),
            time_window_width=np.timedelta64(60, "s"),
            sc="sc1"
        ):
        # Identify collection name to use
        collection = "SW_OPER_MOD{spacecraft}_SC_1B"
        if mission=="Swarm":
            collection = collection.format(spacecraft=spacecraft)
        else:
            raise ValueError(f"{mission} not supported")
        # Identify time window
        start_time = str(time - time_window_width/2)
        end_time = str(time + time_window_width/2)
        # Fetch data from VirES
        request = SwarmRequest(SERVER_URL)
        request.set_collection(collection)
        request.set_products()
        data = request.get_between(start_time, end_time, asynchronous=False, show_progress=False)
        ds = data.as_xarray()
        # Create or append to the existing data as needed
        if self.data[sc]:
            _ds = self.data[sc]
            self.data[sc] = xr.concat([_ds, ds], dim="Timestamp", join="outer")
        else:
            self.data[sc] = ds
        return self.data[sc]
        
    def update_preview(
            self,
            mission1=None, spacecraft1=None,
            mission2=None, spacecraft2=None,
            times=[]
        ):
        for time in times:
            time = np.datetime64(str(time))
            ds1 = self.get_preview_data(
                mission=mission1, spacecraft=spacecraft1, time=time, sc="sc1"
            )
            ds2 = self.get_preview_data(
                mission=mission2, spacecraft=spacecraft2, time=time, sc="sc2"
            )
        points1 = dict(ds1[["Latitude", "Longitude"]])
        points2 = dict(ds2[["Latitude", "Longitude"]])
        self.preview_map.Points.I.data = points1
        self.preview_map.Points.II.data = points2
    
    def reset(self):
        self.data = {"sc1": None, "sc2": None}
    

In [ ]:
preview_backend = DataPreview()
widgets["fetch-previews"] = pn.widgets.Button(name="Preview positions ⬇️", button_type="primary")
widgets["preview-pane"] = pn.pane.HoloViews(preview_backend.display())

def fetch_previews(event):
    widgets["spinner"].value = True
    preview_backend.reset()
    # Identify selected times
    conjs = widgets["conj-times"].value
    indexes = widgets["conj-times"].selection
    times = list(conjs.iloc[indexes].index)
    # Update the preview map with the positional data from those times
    preview_backend.update_preview(
        mission1=widgets["mission1"].value, spacecraft1=widgets["spacecraft1"].value,
        mission2=widgets["mission2"].value, spacecraft2=widgets["spacecraft2"].value,
        times=times
    )
    widgets["preview-pane"].object = preview_backend.display()
    widgets["spinner"].value = False
    
widgets["fetch-previews"].on_click(fetch_previews)

dashboard = pn.layout.Row(
    pn.layout.Column(
        widgets["times"],
        widgets["threshold"],
        pn.layout.Row(
            widgets["mission1"],
            widgets["spacecraft1"],
            width=300
        ),
        pn.layout.Row(
            widgets["mission2"],
            widgets["spacecraft2"],
            width=300
        ),
        widgets["fetch-conjs"],
        widgets["spinner"],
        width=300
    ),
    widgets["conj-times"],
    pn.layout.Column(
        widgets["fetch-previews"],
        widgets["preview-pane"]
    )
)

In [ ]:
dashboard.servable(title="Conjunction Finder")